In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import os
from shapely.geometry.multipolygon import MultiPolygon

In [2]:
os.chdir("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Получение графа и изохрон/isochrones_geopackages")

five_minutes_iso = gpd.read_file("five_minutes_iso.gpkg")
ten_minutes_iso = gpd.read_file("ten_minutes_iso.gpkg")
fifteen_minutes_iso = gpd.read_file("fifteen_minutes_iso.gpkg")
twenty_minutes_iso = gpd.read_file("twenty_minutes_iso.gpkg")

In [ ]:
def get_poi(polygon, tags, **params):
    '''
    Данная функция выгружает из OSM точечные и полигональные объекты в границах входного полигона. \n
    Аргументы функции: \n
    *polygon: Shapely.Polygon или Shapely.MultiPolygon (границы выгрузки) \n
    *tags: dict (тэги объектов) \n
    geometry: str ("Point" - точечные объекты, "Polygon" - полигональные объекты, по умолчанию "Point") \n
    export: bool (True - экспортировать результат в GeoPackage, по умолчанию False)
    file_name: str (название файла, по умолчанию "export")
    '''
    poi = ox.features_from_polygon(polygon, tags=tags).reset_index(drop=True)
    geometry_error = "Ошибка! Недопустимая геометрия."
    export_error = "Ошибка! Задано имя экспортируемого файла, но экспорт не разрешён."

    # Выгрузка геометрии
    if ("geometry" in params and params["geometry"]=="Point") or ("geometry" not in params):
        poi.geometry = poi.geometry.centroid
    elif "geometry" in params and params["geometry"]=="Polygon":
        polygons = poi[poi.geometry.type.isin(['Polygon', 'MultiPolygon'])]
        if len(polygons) == 0:
            print(geometry_error)
        else:
            poi = polygons
            intersector = gpd.GeoDataFrame({'geometry': [polygon]}, crs=poi.crs)
            poi = gpd.overlay(poi, intersector, how='intersection')
            poi["area"] = poi.area
            poi = poi.sort_values("area", ascending=False)
            mean_area = poi['area'].mean()
            std_dev = poi['area'].std() # Избавляемся от выбросов в виде слишком больших/маленьких полигонов
            # (к примеру, весь исторический центр СПБ являлся одним объектом типа park)
            upper_threshold = mean_area + 10 * std_dev
            lower_threshold = mean_area - 10 * std_dev
            poi = poi[(poi['area'] >= lower_threshold) & (poi['area'] <= upper_threshold)]
    else:
        print(geometry_error)
        return
    
    # Настройки экспорта
    if "export" not in params and "file_name" in params:
        print(export_error)
        return
    if "export" in params and params["export"]==True:
        if "file_name" in params and params["file_name"]==None:
            poi.to_file("export.gpkg")
        else:
            poi.to_file(f"{params["file_name"]}.gpkg")
    

    return poi

In [20]:
os.chdir("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Выгрузка POI")

download_mask = twenty_minutes_iso.dissolve().geometry.iloc[0]



In [ ]:
osm_food_amenities_tags = {
    "amenity": ["restaurant", 
                "fast_food", 
                "cafe", "food_court", 
                "ice_cream", 
                "biergarten" ]
}

osm_food_amenities = get_poi(download_mask, 
                              osm_food_amenities_tags, 
                              geometry="Point", 
                              export=True, 
                              file_name="food")


C:\Users\user\AppData\Local\Temp\ipykernel_3284\4014514692.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi.geometry = poi.geometry.centroid


In [ ]:
osm_leisure_tags = {"amenity": ["cinema", "nightclub", "community_centre", "bar", "pub"],
                    "leisure": ["amusement_park", "dance", "escape_game", "miniature_golf", "bowling_alley", "water_park", "sports_centre"]}
osm_leisure = get_poi(download_mask, 
                    osm_leisure_tags, 
                    geometry="Point", 
                    export=True, 
                    file_name="leisure")

C:\Users\user\AppData\Local\Temp\ipykernel_3284\4014514692.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi.geometry = poi.geometry.centroid


In [ ]:
osm_tourist_tags = {"amenity":["theater", "arts_center"],
                    "tourism":True}
osm_tourism = get_poi(download_mask, 
                    osm_tourist_tags, 
                    geometry="Point", 
                    export=True, 
                    file_name="tourism")

C:\Users\user\AppData\Local\Temp\ipykernel_3284\4014514692.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi.geometry = poi.geometry.centroid


In [ ]:
osm_shop_tags = {
    "shop": ["supermarket", "convenience", "clothes", "shoes", "electronics", "furniture", "books", "sports", "jewelry", "pharmacy", "bakery", "butcher", "greengrocer", "florist", "optician", "beauty", "hardware", "computer", "toys", "alcohol", "mobile_phone", "stationery", "gift", "mall", "outdoor"]
}
osm_shops = get_poi(download_mask, 
                    osm_shop_tags, 
                    geometry="Point", 
                    export=True, 
                    file_name="shops")

C:\Users\user\AppData\Local\Temp\ipykernel_3284\4014514692.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi.geometry = poi.geometry.centroid


In [ ]:
osm_greenery_tags = {
    "leisure": ["park", "garden", "nature_reserve"],
    "landuse": ["forest", "meadow", "grass", "allotments", "orchard", "vineyard"],
    "natural": ["wood", "scrub", "heath", "grassland", "wetland"],
    "boundary": ["protected_area"],
    "tourism": ["botanical_garden", "zoo"]
}
osm_greenery = get_poi(download_mask, 
                    osm_greenery_tags, 
                    geometry="Polygon", 
                    export=True, 
                    file_name="greenery")

C:\Users\user\AppData\Local\Temp\ipykernel_3284\916660873.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi["area"] = poi.area
